In [37]:
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook as tqdm

#set the max columns to none"
pd.set_option('display.max_columns', None)

### I will create the interaction matrix from history data and save it in order to quick access

In [110]:
size = 'demo'
type_ = 'test'
#Import train Data
history_train_path = f'./files/parquet/ebnerd_{size}/{type_}/history.parquet'

history_df = pd.read_parquet(history_train_path)
print(history_df.shape)
try:
    history_df.drop('impression_time_fixed', axis=1, inplace=True) # i will drop this column initialy even though i can proccess it and use it
except:
    pass

behaviors_path = f'./files/parquet/ebnerd_{size}/{type_}/behaviors.parquet'
behaviors_df = pd.read_parquet(behaviors_path)
print(f'Raw {size} validation behaviors df shape:          ',behaviors_df.shape)


(469, 5)
Raw demo validation behaviors df shape:           (7607, 17)


In [111]:
# from sklearn.model_selection import train_test_split

# val_df, test_df = train_test_split(history_df, test_size=0.3, random_state=42)

# # Save the splits as Parquet files
# test_path = "./files/parquet/ebnerd_demo/test/history.parquet"
# val_path = "./files/parquet/ebnerd_demo/validation/history.parquet"

# test_df.to_parquet(test_path, index=False)
# val_df.to_parquet(val_path, index=False)

In [112]:

def replace_nan_with_zero_in_column(df, column_name):
    """
    Function to replace NaN values in a specific column with 0.
    
    Parameters:
    - df: Pandas DataFrame
    - column_name: Name of the column in which to replace NaN values with 0
    
    Returns:
    - df: Updated DataFrame with NaN values replaced by 0 in the specified column
    """
    df[column_name].fillna(0, inplace=True)
    return df

# Apply the function to replace NaN with 0 in 'read_time_fixed' column
behaviors_df = replace_nan_with_zero_in_column(behaviors_df, 'scroll_percentage')
behaviors_df = replace_nan_with_zero_in_column(behaviors_df, 'read_time')



In [113]:
# def replace_nans_with_empty_lists(df, column_names):
#     """
#     Replaces NaN values in specified DataFrame columns with empty lists.

#     Parameters:
#     df (pd.DataFrame): The DataFrame containing the columns.
#     column_names (list of str): The names of the columns to process.

#     Returns:
#     pd.DataFrame: The DataFrame with NaNs replaced by empty lists in the specified columns.
#     """
#     for column in column_names:
#         def replace_nan(x):
#             if isinstance(x, (list, np.ndarray)):
#                 # If x is a list or array, check for any NaNs within
#                 if any(pd.isnull(e) for e in x):
#                     return []
#                 else:
#                     return x
#             else:
#                 # If x is not a list or array, check if it is NaN
#                 return [] if pd.isnull(x) else x
        
#         df[column] = df[column].apply(replace_nan)
#     return df

# columns_to_replace = ['article_ids_clicked', 'read_time', 'scroll_percentage'] #
# final_df = replace_nans_with_empty_lists(final_df, columns_to_replace)

In [114]:
history_df.head(2)

,user_id,scroll_percentage_fixed,article_id_fixed,read_time_fixed
0,1795319,"[nan, 16.0, nan, nan, nan]","[9775596, 9775596, 9775596, 9775596, 9777856]","[597.0, 1568.0, 0.0, 2.0, 0.0]"
1,1948909,"[100.0, 49.0, 100.0, 66.0, 100.0, 53.0, 100.0,...","[9754484, 9754271, 9754484, 9754374, 9754484, ...","[191.0, 13.0, 5.0, 19.0, 4.0, 18.0, 8.0, 10.0,..."


In [115]:
behaviors_df.head(2)

,impression_id,article_id,impression_time,read_time,scroll_percentage,device_type,article_ids_inview,article_ids_clicked,user_id,is_sso_user,gender,postcode,age,is_subscriber,session_id,next_read_time,next_scroll_percentage
0,120499774,9784444.0,2023-05-27 17:43:13,104.0,100.0,2,"[9709817, 9666599, 9784444, 9783852, 9671904, ...",[9671904],1744238,True,0.0,NaN,NaN,True,191793,405.0,100.0
1,156353986,NaN,2023-05-30 10:54:38,41.0,0.0,2,"[9788134, 9788239, 9788125, 9788225, 9788145]",[9788225],2436485,False,NaN,NaN,NaN,False,68070,93.0,100.0


In [116]:
# Preprocess behaviour
behaviors_df = behaviors_df[['article_ids_clicked','user_id','read_time','scroll_percentage']]
behaviors_df = behaviors_df.explode('article_ids_clicked')

# behaviors_df.groupby('user_id')['article_ids_clicked'].apply(list).reset_index()

# behaviors_df.groupby('user_id')['read_time'].apply(list).reset_index()

behaviors_df_merged= pd.merge(behaviors_df.groupby('user_id')['article_ids_clicked'].apply(list).reset_index(),behaviors_df.groupby('user_id')['read_time'].apply(list).reset_index(),on='user_id')

behaviors_df= pd.merge(behaviors_df_merged,behaviors_df.groupby('user_id')['scroll_percentage'].apply(list).reset_index(),on='user_id')


In [117]:
behaviors_df.head()

,user_id,article_ids_clicked,read_time,scroll_percentage
0,19181,"[9780267, 9782519, 6741781, 9780677, 9788947, ...","[19.0, 5.0, 19.0, 11.0, 57.0, 248.0, 15.0, 13.0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
1,21271,"[9339920, 9784097]","[30.0, 83.0]","[0.0, 100.0]"
2,22779,"[9788666, 9785111, 9784344, 9782806]","[9.0, 14.0, 11.0, 10.0]","[0.0, 0.0, 0.0, 0.0]"
3,22895,"[9786222, 9784393, 9790574, 9786213, 9783852, ...","[9.0, 6.0, 5.0, 31.0, 8.0, 9.0, 10.0, 9.0, 27....","[0.0, 100.0, 0.0, 100.0, 0.0, 0.0, 0.0, 100.0,..."
4,24184,"[9783236, 9788151, 9780861, 9781523, 9789922, ...","[52.0, 12.0, 16.0, 40.0, 14.0, 33.0, 59.0, 23....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [118]:
history_df.shape

(469, 4)

In [119]:
# history_df['scroll_percentage_fixed_len'] = history_df['scroll_percentage_fixed'].apply(lambda lst: len(lst))
# history_df['article_id_fixed_len'] = history_df['article_id_fixed'].apply(lambda lst: len(lst))
# history_df['read_time_fixed_len'] = history_df['read_time_fixed'].apply(lambda lst: len(lst))

# history_df['scroll_percentage_fixed_has_0_in'] = history_df['scroll_percentage_fixed'].apply(lambda lst: 0 in lst)
# history_df['read_time_fixed_has_0_in'] = history_df['read_time_fixed'].apply(lambda lst: 0 in lst)
# history_df['read_time_fixed_has_nan'] = history_df['read_time_fixed'].apply(lambda lst: pd.isna(lst.any()))

In [120]:
# # Combine interactions into a single metric (Potential use of scroll_percentage_fixed column)
# def combine_interactions(interactions, interactions2, w1=0.5, w2=0.5):
#     return [w1*i + w2*i2  for i, i2 in zip(interactions, interactions2)]

# history_dfraction_combined'] = history_df(lambda row:   combine_interactions(row['scroll_percentage_fixed'], row['read_time_fixed']) , axis=1)

# history_df ['scroll_percentage_fixed', 'read_time_fixed'], axis=1, inplace=True)

In [121]:
# history_df.drop(['scroll_percentage_fixed_len', 'article_id_fixed_len', 'read_time_fixed_len', 'scroll_percentage_fixed_has_0_in', 'read_time_fixed_has_nan' ,'read_time_fixed_has_0_in'], axis=1, inplace=True)

In [122]:
## Merge behaviours and history-left join
final_df = pd.merge(history_df,behaviors_df,on='user_id',how='outer')

In [123]:
def convert_arrays_to_lists(df, column_name):
    """
    Function to convert arrays (or other iterable objects) in a column to lists.
    
    Parameters:
    - df: Pandas DataFrame
    - column_name: Name of the column containing arrays or other iterables
    
    Returns:
    - df: Updated DataFrame with the column converted to lists
    """
    df[column_name] = df[column_name].apply(lambda x: x.tolist() if isinstance(x, np.ndarray) else x)
    return df

# Convert arrays in 'data_arrays' column to lists
for col in final_df.columns: #['scroll_percentage_fixed', 'article_id_fixed', 'read_time_fixed']
    final_df = convert_arrays_to_lists(final_df, col)

In [124]:
def replace_nans_with_empty_lists(df, column_names):
    """
    Replaces NaN values in specified DataFrame columns with empty lists.

    Args:
    df (pd.DataFrame): The input DataFrame.
    column_names (list of str): The column names in which to replace NaNs with empty lists.

    Returns:
    pd.DataFrame: The DataFrame with NaNs replaced by empty lists in the specified columns.
    """
    for column in column_names:
        df[column] = df[column].apply(lambda x: x if isinstance(x, list) else [])
    return df
# Apply the function to replace NaN with 0 in 'read_time_fixed' column
columns_to_replace = ['article_ids_clicked', 'read_time', 'scroll_percentage']

final_df = replace_nans_with_empty_lists(final_df, columns_to_replace)


In [125]:
# final_df['len'] = final_df['read_time'].apply(lambda x: len(x))

In [126]:
final_df.head()

,user_id,scroll_percentage_fixed,article_id_fixed,read_time_fixed,article_ids_clicked,read_time,scroll_percentage
0,1795319,"[nan, 16.0, nan, nan, nan]","[9775596, 9775596, 9775596, 9775596, 9777856]","[597.0, 1568.0, 0.0, 2.0, 0.0]","[9782360, 9784782]","[44.0, 47.0]","[100.0, 0.0]"
1,1948909,"[100.0, 49.0, 100.0, 66.0, 100.0, 53.0, 100.0,...","[9754484, 9754271, 9754484, 9754374, 9754484, ...","[191.0, 13.0, 5.0, 19.0, 4.0, 18.0, 8.0, 10.0,...","[9277339, 9781947]","[13.0, 17.0]","[0.0, 0.0]"
2,2098487,"[100.0, 32.0, 42.0, 20.0, 100.0, 100.0, nan, 1...","[9749677, 9749584, 9749644, 9749627, 9749454, ...","[20.0, 6.0, 4.0, 5.0, 71.0, 50.0, 2.0, 48.0, 5...","[9789616, 9787261, 9780514, 9780580, 9780498, ...","[40.0, 9.0, 23.0, 35.0, 149.0, 34.0, 13.0, 6.0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,1428063,"[30.0, 44.0, 100.0, 93.0, 38.0, 34.0, 87.0, 87...","[9749601, 9753545, 9753584, 9753901, 9753727, ...","[10.0, 15.0, 15.0, 12.0, 5.0, 6.0, 8.0, 28.0, ...",[],[],[]
4,458124,"[20.0, 79.0, 97.0, 19.0, 29.0, 100.0, 74.0, 94...","[9749729, 9749495, 9749247, 9749601, 9749349, ...","[5.0, 68.0, 84.0, 5.0, 10.0, 21.0, 40.0, 70.0,...","[9789623, 9785009, 9782027, 9786821, 9783646, ...","[10.0, 6.0, 61.0, 12.0, 14.0, 15.0, 15.0, 21.0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [127]:
def extend_lists_in_columns(df, col1, col2, new_col_name):
    """
    Extends the list in each cell of the second column with the list in the first column
    and stores the result in a new column.

    Args:
    df (pd.DataFrame): The input DataFrame.
    col1 (str): The name of the first column containing lists.
    col2 (str): The name of the second column containing lists.
    new_col_name (str): The name of the new column to store the extended lists.

    Returns:
    pd.DataFrame: The DataFrame with the new column containing the extended lists.
    """
    def extend_lists(row):
        list1 = row[col1] if isinstance(row[col1], list) else []
        list2 = row[col2] if isinstance(row[col2], list) else []
        return list1 + list2

    df[new_col_name] = df.apply(extend_lists, axis=1)
    return df
    
# def combine_read_time_lists(row):
#     if row['read_time_fixed']:  # Check if article_ids_clicked is not empty
#         return list(row['read_time_fixed']) + list(row['read_time'])
#     else:
#         return list(row['read_time_fixed'])
    
# def combine_scroll_percentage_lists(row):
#     if row['scroll_percentage_fixed']:  # Check if article_ids_clicked is not empty
#         return list(row['scroll_percentage_fixed']) + list(row['scroll_percentage'])
#     else:
#         return list(row['scroll_percentage_fixed'])

In [128]:
# # Convert potential NaN values to empty lists for safe concatenation
# final_df['article_id_fixed'] = final_df['article_id_fixed'].apply(lambda x: x if isinstance(x, list) else [])
# final_df['article_ids_clicked'] = final_df['article_ids_clicked'].apply(lambda x: x if isinstance(x, list) else [])


In [129]:
# Combine article_id_fixed and article_ids_clicked into a single column
final_df = extend_lists_in_columns(final_df, 'article_id_fixed', 'article_ids_clicked', 'merged_article_ids')
final_df = extend_lists_in_columns(final_df, 'scroll_percentage_fixed', 'scroll_percentage', 'merged_scroll_percentage')
final_df = extend_lists_in_columns(final_df, 'read_time_fixed', 'read_time', 'merged_read_time')

In [130]:
final_df.shape

(1363, 10)

In [131]:
final_df.head()

,user_id,scroll_percentage_fixed,article_id_fixed,read_time_fixed,article_ids_clicked,read_time,scroll_percentage,merged_article_ids,merged_scroll_percentage,merged_read_time
0,1795319,"[nan, 16.0, nan, nan, nan]","[9775596, 9775596, 9775596, 9775596, 9777856]","[597.0, 1568.0, 0.0, 2.0, 0.0]","[9782360, 9784782]","[44.0, 47.0]","[100.0, 0.0]","[9775596, 9775596, 9775596, 9775596, 9777856, ...","[nan, 16.0, nan, nan, nan, 100.0, 0.0]","[597.0, 1568.0, 0.0, 2.0, 0.0, 44.0, 47.0]"
1,1948909,"[100.0, 49.0, 100.0, 66.0, 100.0, 53.0, 100.0,...","[9754484, 9754271, 9754484, 9754374, 9754484, ...","[191.0, 13.0, 5.0, 19.0, 4.0, 18.0, 8.0, 10.0,...","[9277339, 9781947]","[13.0, 17.0]","[0.0, 0.0]","[9754484, 9754271, 9754484, 9754374, 9754484, ...","[100.0, 49.0, 100.0, 66.0, 100.0, 53.0, 100.0,...","[191.0, 13.0, 5.0, 19.0, 4.0, 18.0, 8.0, 10.0,..."
2,2098487,"[100.0, 32.0, 42.0, 20.0, 100.0, 100.0, nan, 1...","[9749677, 9749584, 9749644, 9749627, 9749454, ...","[20.0, 6.0, 4.0, 5.0, 71.0, 50.0, 2.0, 48.0, 5...","[9789616, 9787261, 9780514, 9780580, 9780498, ...","[40.0, 9.0, 23.0, 35.0, 149.0, 34.0, 13.0, 6.0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[9749677, 9749584, 9749644, 9749627, 9749454, ...","[100.0, 32.0, 42.0, 20.0, 100.0, 100.0, nan, 1...","[20.0, 6.0, 4.0, 5.0, 71.0, 50.0, 2.0, 48.0, 5..."
3,1428063,"[30.0, 44.0, 100.0, 93.0, 38.0, 34.0, 87.0, 87...","[9749601, 9753545, 9753584, 9753901, 9753727, ...","[10.0, 15.0, 15.0, 12.0, 5.0, 6.0, 8.0, 28.0, ...",[],[],[],"[9749601, 9753545, 9753584, 9753901, 9753727, ...","[30.0, 44.0, 100.0, 93.0, 38.0, 34.0, 87.0, 87...","[10.0, 15.0, 15.0, 12.0, 5.0, 6.0, 8.0, 28.0, ..."
4,458124,"[20.0, 79.0, 97.0, 19.0, 29.0, 100.0, 74.0, 94...","[9749729, 9749495, 9749247, 9749601, 9749349, ...","[5.0, 68.0, 84.0, 5.0, 10.0, 21.0, 40.0, 70.0,...","[9789623, 9785009, 9782027, 9786821, 9783646, ...","[10.0, 6.0, 61.0, 12.0, 14.0, 15.0, 15.0, 21.0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[9749729, 9749495, 9749247, 9749601, 9749349, ...","[20.0, 79.0, 97.0, 19.0, 29.0, 100.0, 74.0, 94...","[5.0, 68.0, 84.0, 5.0, 10.0, 21.0, 40.0, 70.0,..."


In [132]:
final_df = final_df[['merged_article_ids','merged_read_time','user_id']]

In [133]:
final_df.head(2)

,merged_article_ids,merged_read_time,user_id
0,"[9775596, 9775596, 9775596, 9775596, 9777856, ...","[597.0, 1568.0, 0.0, 2.0, 0.0, 44.0, 47.0]",1795319
1,"[9754484, 9754271, 9754484, 9754374, 9754484, ...","[191.0, 13.0, 5.0, 19.0, 4.0, 18.0, 8.0, 10.0,...",1948909


In [134]:
# final_df.set_index('user_id',inplace=True)

In [135]:
# final_df = final_df.explode(['merge_scroll_percentage','merge_article_ids']).dropna(subset='merge_scroll_percentage')

In [136]:
# final_df.drop(axis=1,inplace=True,columns='merge_scroll_percentage')

In [137]:
final_df.head()

,merged_article_ids,merged_read_time,user_id
0,"[9775596, 9775596, 9775596, 9775596, 9777856, ...","[597.0, 1568.0, 0.0, 2.0, 0.0, 44.0, 47.0]",1795319
1,"[9754484, 9754271, 9754484, 9754374, 9754484, ...","[191.0, 13.0, 5.0, 19.0, 4.0, 18.0, 8.0, 10.0,...",1948909
2,"[9749677, 9749584, 9749644, 9749627, 9749454, ...","[20.0, 6.0, 4.0, 5.0, 71.0, 50.0, 2.0, 48.0, 5...",2098487
3,"[9749601, 9753545, 9753584, 9753901, 9753727, ...","[10.0, 15.0, 15.0, 12.0, 5.0, 6.0, 8.0, 28.0, ...",1428063
4,"[9749729, 9749495, 9749247, 9749601, 9749349, ...","[5.0, 68.0, 84.0, 5.0, 10.0, 21.0, 40.0, 70.0,...",458124


In [138]:
final_df = final_df.explode('merged_article_ids')

In [139]:
final_df.shape

(78156, 3)

In [140]:
# Creation of interaction matrix

interaction_matrix = pd.DataFrame(columns=final_df.columns)
counter = 0
for user_id in tqdm(np.unique(final_df.index.values).tolist()):
    
    temp_df = final_df[final_df.index == user_id].copy()
    temp_df = temp_df.explode('merged_article_ids')
    temp_df.set_index('user_id',inplace=True)
    temp_lst = []

    # Fix the lists that exist in each cell and keep the values that i care
    temp_counter = 0
    for lst in list(temp_df['merged_read_time']):
        temp_lst.append(lst[temp_counter])
        temp_counter += 1
    
    temp_df['merged_read_time_fixed'] = temp_lst

    # Drop first unneccesary column
    temp_df.drop('merged_read_time', axis=1, inplace= True)
    temp_pivoted_df = temp_df.pivot_table(index = temp_df.index, columns='merged_article_ids', values='merged_read_time_fixed', aggfunc='sum').copy()

    if counter == 0:
        interaction_matrix = temp_pivoted_df.copy()
        #print(interaction_matrix)
    else:
        interaction_matrix = pd.concat([interaction_matrix, temp_pivoted_df], axis=0, ignore_index=False)
    
    counter += 1

C:\Users\Intel PC G5900\AppData\Local\Temp\ipykernel_15332\3818264614.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for user_id in tqdm(np.unique(final_df.index.values).tolist()):


  0%|          | 0/1363 [00:00<?, ?it/s]

In [141]:
interaction_matrix.shape

(1363, 3940)

In [142]:
interaction_matrix.head()

merged_article_ids  9775596  9777856  9782360  9784782  9277339  9695098  \
user_id                                                                    
1795319              2167.0      0.0     44.0     47.0      NaN      NaN   
1948909                 NaN      NaN      NaN      NaN     13.0      3.0   
2098487                 NaN      NaN      NaN      NaN      NaN      NaN   
1428063                 NaN      NaN      NaN      NaN      NaN      NaN   
458124                  NaN      NaN      NaN      NaN      NaN      NaN   

merged_article_ids  9747961  9752741  9753811  9754081  9754149  9754269  \
user_id                                                                    
1795319                 NaN      NaN      NaN      NaN      NaN      NaN   
1948909                10.0     10.0      7.0      8.0     17.0     18.0   
2098487                 NaN      NaN      NaN      NaN      NaN      NaN   
1428063                 NaN      NaN      NaN     53.0      NaN      NaN   
458124                 48.0      NaN      NaN      NaN      NaN     82.0   

merged_article_ids  9754271  9754374  9754484  9755656  9755726  9755964  \
user_id                                                                    
1795319                 NaN      NaN      NaN      NaN      NaN      NaN   
1948909                13.0     19.0    248.0    101.0      5.0      6.0   
2098487                 NaN      NaN      NaN    154.0      NaN      NaN   
1428063                 NaN      NaN      NaN      NaN      NaN      8.0   
458124                  NaN      NaN     47.0    111.0      NaN     94.0   

merged_article_ids  9756047  9756107  9756362  9757181  9757224  9757425  \
user_id                                                                    
1795319                 NaN      NaN      NaN      NaN      NaN      NaN   
1948909                14.0      5.0   1200.0     15.0      3.0     89.0   
2098487                 NaN      NaN     28.0      NaN      NaN      NaN   
1428063                 NaN      NaN     10.0      NaN      NaN      NaN   
458124                 33.0     24.0     54.0      NaN     30.0      0.0   

merged_article_ids  9757552  9757592  9757920  9758432  9759154  9759309  \
user_id                                                                    
1795319                 NaN      NaN      NaN      NaN      NaN      NaN   
1948909                25.0      4.0     65.0     15.0     14.0     37.0   
2098487                 NaN      NaN    113.0      NaN      NaN      NaN   
1428063                 NaN      NaN      NaN      NaN      NaN      NaN   
458124                  NaN      NaN      NaN      NaN      NaN      NaN   

merged_article_ids  9759355  9759398  9759433  9759482  9759595  9759707  \
user_id                                                                    
1795319                 NaN      NaN      NaN      NaN      NaN      NaN   
1948909               343.0     26.0     13.0     53.0      4.0     85.0   
2098487                68.0      NaN      NaN      NaN     53.0      NaN   
1428063                 NaN      NaN      NaN      NaN      NaN      NaN   
458124                  NaN      NaN      NaN      NaN      NaN      NaN   

merged_article_ids  9761005  9761359  9761620  9761638  9761683  9761803  \
user_id                                                                    
1795319                 NaN      NaN      NaN      NaN      NaN      NaN   
1948909                21.0      4.0      4.0      6.0     13.0     60.0   
2098487                 NaN      NaN      NaN      NaN      NaN      NaN   
1428063                 NaN     31.0      7.0     11.0      NaN      NaN   
458124                  NaN      7.0      NaN      NaN      NaN     38.0   

merged_article_ids  9761862  9765894  9766129  9766146  9766178  9766307  \
user_id                                                                    
1795319                 NaN      NaN      NaN      NaN      NaN      NaN   
1948909                 6.0    962.0     

In [143]:
fillna_value = '0'
interaction_matrix.fillna(0, inplace=True)

In [144]:
fillna_value

'0'

In [145]:
# Save the interaction matrix to pickle
interaction_matrix_file_path = f'./files/pickle/interaction_matrix_{type_}_{size}_{fillna_value}.pkl'

interaction_matrix.to_pickle(interaction_matrix_file_path)

In [200]:
# Read interaction matrix pickle file
interaction_matrix_file_path = f'./files/pickle/interaction_matrix_{size}_{fillna_value}.pkl'
interaction_matrix = pd.read_pickle(interaction_matrix_file_path)

FileNotFoundError: [Errno 2] No such file or directory: './files/pickle/interaction_matrix_demo_0.pkl'

## Other

In [ ]:
# # Combine interactions into a single metric
# def combine_interactions(interactions, interactions2, w1=1, w2=1):
#     return [w1*i + w2*i2  for i, i2 in zip(interactions, interactions2)]

# # Flatten the interaction data with combined interactions
# flattened_data = []

# for _, row in history_df.iterrows():
#     user_id = row['user_id']
#     article_ids = row['article_id_fixed']
#     combined_interactions = combine_interactions(row['scroll_percentage_fixed'], row['read_time_fixed'])
    
#     for article_id, interaction_combined in zip(article_ids, combined_interactions):
#         flattened_data.append({
#             'user_id': user_id, 
#             'article_id_fixed': article_id, 
#             'interaction_combined': interaction_combined
#         })

# # Convert to DataFrame
# interaction_df = pd.DataFrame(flattened_data)